# Compare the model likelihood of the same sentence under two models.

> :warning: **Environment**: ipython has a lot of dependencies and is not in the main training environment. Hence this needs to run under a seperate one.


In [ ]:
import os
from typing import Dict, Tuple, List
import itertools
import sys
import rich
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from omegaconf import OmegaConf
import numpy as np
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm

sys.path.append(os.path.dirname(os.getcwd()))

from notebooks.utils import styles




## Load data

Load data from pickle files and store in DF.

In [ ]:
# SETUP THESE VARIABLES
run_names = ["6e43-10b0", "6385-5556"]
# removes the most likely samples that are not actually clearly OOD
REMOVE_TOP_SAMPLES_G_LIKELIHOOD = 36 # start off with 0, and print the laoded data below. Find how many of the top samples in the D_F data are not clearly OOD (datasets are not perfectly clean). Then set that number here.
# Set this name used for plotting and saving figures. e.g. "MedicalQA"
EXPERIMENT_NAME = ""

##### LOAD BY RUN NAMES

paths = [
    f"/path/to/your/artifacts/model_likelihood/{run_name}/model_likelihood.pkl" for run_name in run_names
]

print(paths)
print(f"Loaded {len(paths)} likelihod outputs.")

# in the data you load, datasets have names. specify which names are ID and which are OOD. Not all have to be in the dataset at the same time.
DOMAIN = "MedicalQA"
ID_NAMES = ["pubmedqa", "pubmedqa_generated"]


# we remove all samples shorter than a threshold as they get very ambiguous whether they are ID or OOD.
MIN_LENGTH = 10
MAX_LENGTH = None


In [ ]:

def load_experiment_data(paths: List[str]) -> List[Dict]:
    experiments = []

    for path in paths:
        try:
            with open(path, 'rb') as f:
                experiments.append(pickle.load(f))
        except:
            print(f"Could not load: {path}")

    # print laoded data
    for i, experiment in enumerate(experiments):
        print(experiment["config"]["data"]["name"])
        print(experiment["config"]["model"]["name_or_path"])
        print(experiment["config"]["generator"]["name_or_path"])

    # compute the log likelihoods
    records = []
    for i, experiment in enumerate(experiments):
        data = experiment['data']
        config = OmegaConf.to_container(experiment['config'])

        # obtain the tokenizer and model from the config when tokenizers vary
        if "n_token_prompt" not in data and config["tokenizers_match"]:
            data["n_token_prompt"] = data["n_token_prompt_model"]
            data["sequence_length"] = data["sequence_length_model"]
            data["x_text"] = data["x_text_model"]
            data["y_text"] = data["y_text_model"]
        else:
            raise ValueError("No token prompt found.")

        N_samples = data["n_token_prompt"].shape[0]
        ll_model = data["log_likelihoods_model"].sum(-1)
        ll_generator = data["log_likelihoods_generator"].sum(-1)
        ll2_model = ll_model / np.log(2)
        ll2_generator = ll_generator / np.log(2)
        ll10_model = ll_model / np.log(10)
        ll10_generator = ll_generator / np.log(10)
        n_token_response =  data["sequence_length"] - data["n_token_prompt"].squeeze()

        # loge_ratio = ll_model - ll_generator
        log2_ratio = ll2_model - ll2_generator
        norm_log2_ratio = log2_ratio / n_token_response

        config_columns = pd.json_normalize(config)
        dist_F = config_columns['model.target_distribution'][0]
        dist_G = config_columns['generator.target_distribution'][0]

        entropy_model = data["entropy_model"].sum(-1)
        entropy_generator = data["entropy_generator"].sum(-1)

        config_columns["distributions"] = f"F({dist_F})||G({dist_G})"

        if config["inference"]["prompt_length"] == "dataset":
            prompt_length = data["n_token_prompt"].squeeze()
        else:
            prompt_length = np.full((N_samples,), int(config["inference"]["prompt_length"]))

        n_char_response = [len(x) for x in data["y_text"]]

        data_columns = pd.DataFrame({
            "ll2_model": ll2_model,
            "ll2_generator": ll2_generator,
            "ll10_model": ll10_model,
            "ll10_generator": ll10_generator,
            "ll2_model_norm": ll2_model / n_token_response,
            "ll2_generator_norm": ll2_generator / n_token_response,
            "ll10_model_norm": ll10_model / n_token_response,
            "ll10_generator_norm": ll10_generator / n_token_response,
            "log2_ratio": log2_ratio,
            "log2_ratio_norm": log2_ratio / n_token_response,
            "entropy_model": entropy_model,
            "entropy_generator": entropy_generator,
            "x": data["x_text"],
            "y": data["y_text"],
            "sequence_length": data["sequence_length"],
            "n_token_prompt": data["n_token_prompt"].squeeze(),
            "prompt_length": prompt_length,
            "n_token_response": n_token_response,
            "n_char_response": n_char_response,
        })

        assert data_columns["n_token_response"].min() > 0

        # combine and repeat config_columns to match data_columns
        config_columns = pd.concat([config_columns] * len(data_columns), ignore_index=True)
        combined = pd.concat([config_columns, data_columns], axis=1)

        records.append(combined)

    df = pd.concat(records).copy()

    # OOD is when data_config_name is not task_data_int_sort
    df["OOD"] = (~df["data_config_name"].isin(ID_NAMES)).astype(int)
    df["Dataset"] = df["OOD"].map({0: r"Target Domain $\mathcal{D}_{T}$", 1: r"Other $\mathcal{D}_{F}$"})

    return df


df = load_experiment_data(paths)

# print unique combinations of distributions and data_config_name
df_unique_combinations = df[["distributions", "data_config_name"]].drop_duplicates()
print(df_unique_combinations)
print(df.shape)

print(f"Model M: {df['model.name_or_path'].unique()}")
print(f"Model G: {df['generator.name_or_path'].unique()}")

print(df.groupby(["data_config_name", "distributions"]).size().reset_index(name='counts'))


## Clean Dataset

1. Remove short responses.

In [ ]:
df_original_shape = df.shape
if MIN_LENGTH is not None:
    df = df.loc[df["n_token_response"] >= MIN_LENGTH]
    print(f"Removed {df_original_shape[0] - df.shape[0]} rows with n_token_response < {MIN_LENGTH}. Remaining: {df.shape[0]}")
if MAX_LENGTH is not None:
    df = df.loc[df["n_token_response"] <= MAX_LENGTH]
    print(f"Removed {df_original_shape[0] - df.shape[0]} rows with n_token_response > {MAX_LENGTH}. Remaining: {df.shape[0]}")


2. Remove responses that are not clearly OOD, but could in fact be ID.

In [ ]:
# remove the N samples with the highest `ll2_generator` values as first cleaning stragey.
original_shape = df.shape
df = df.sort_values("ll2_generator", ascending=False)
df = df.iloc[REMOVE_TOP_SAMPLES_G_LIKELIHOOD:] # SET THIS ABOVE.
print(f"Removed {original_shape[0] - df.shape[0]} samples with highest `ll2_generator` values. Remaining: {df.shape[0]}")

# Save df

In [ ]:
# save the data
df.to_csv(f"df_{DOMAIN}{EXPERIMENT_NAME}.csv", index=False, sep="\t", encoding="utf-8")

# General Functions

In [ ]:
def get_log_upper_bound_m(log_g: float, N: int, T: int, k: float) -> float:
    return log_g + np.log2(T) + k * N

def get_k_given_frr(df, frr: float):
    k = df.loc[df["OOD"] == 0, "log2_ratio_norm"].quantile(1 - frr)
    return k

def get_k_given_epsilon(e_log10: float, T: int, df: pd.DataFrame, k_min: float = 0, k_max: float = 10, max_iter: int = 100, tolerance: float = 1e-6, upper_quantile: float = 1.0) -> Tuple[float, float, float]:
    e_log2 = e_log10 / np.log10(2)

    for _ in range(max_iter):
        k_mid = (k_min + k_max) / 2
        # Calculate log_upper_bounds for each row in the DataFrame
        log_upper_bounds = df.apply(lambda row: get_log_upper_bound_m(row["ll2_generator"], row["n_token_response"], T, k_mid), axis=1)
        max_log_upper_bound = log_upper_bounds.quantile(upper_quantile)
        # print(f"Current k: {k_mid}. k_min: {k_min}, k_max: {k_max} | Max log upper bound: {max_log_upper_bound} | e_log2: {e_log2}")

        # Check if the current max_log_upper_bound is within the tolerance
        if abs(max_log_upper_bound - e_log2) < tolerance:
            return (k_min, k_mid, k_max)

        # Adjust the search range based on comparison
        if max_log_upper_bound > e_log2:
            k_max = k_mid
        else:
            k_min = k_mid

    # check solution
    if abs(max_log_upper_bound - e_log2) > tolerance:
        print(f"Solution not found within {max_iter} iterations. Current max_log_upper_bound: {max_log_upper_bound} | e_log2: {e_log2}")

    return (k_min, k_mid, k_max)

def get_constriction_ratios_given_k(df: pd.DataFrame, k: float, T: int, return_base: str = "log10") -> np.ndarray:
    # Calculate log_upper_bounds for each row in the DataFrame
    log2_upper_bounds = df.apply(lambda row: get_log_upper_bound_m(row["ll2_generator"], row["n_token_response"], T, k), axis=1)
    log2_cr = df["ll2_model"] - log2_upper_bounds
    if return_base == "log2":
        return log2_cr.to_numpy()
    if return_base == "log10":
        # log 10 x = log 2 x / log 2 10
        log10_cr = log2_cr / np.log2(10)
        return log10_cr.to_numpy()
    if return_base in ["log", "loge", "ln"]:
        # log e x = log 2 x / log 2 e
        log_cr = log2_cr / np.log2(np.e)
        return log_cr.to_numpy()
    if return_base in ["p", "prob", "probability"]:
        # p(x) = 2 ^ log2(x)
        p_cr = 2 ** log2_cr
        return p_cr.to_numpy()


# Get Table of bounds and thresholds

In [ ]:
# This cell saves a table that is then loaded for the `resuts_benchmark.ipynb` notebook.

T_list = [1]

log10_eps_list = np.linspace(-50, 50, 501)
upper_quantile = 1.0 # quantile for the DC bound. 1.0 is the maximum, < 1.0 might be appropriate, when datasets are clearly not clean and contain ID sequences. We use 1.0 here.

def get_eps_table_from_epsilon(T_list: List[int], log10_eps_list: List[float], df: pd.DataFrame, upper_quantile: float = 1.0) -> pd.DataFrame:
    pars = list(itertools.product(T_list, log10_eps_list))
    results = []

    for T, e_log10 in tqdm(pars):
        k_min, k_mid, k_max = get_k_given_epsilon(e_log10, T, df, k_min=-100, k_max=100, max_iter=100, tolerance=1e-6, upper_quantile=upper_quantile)
        constriction_ratios = get_constriction_ratios_given_k(df, k_mid, T, return_base="log10")
        results.append({
            "T": T,
            "log10_eps": e_log10,
            "k_min": k_min,
            "k_mid": k_mid,
            "k_max": k_max,
            "log10_cr_10": np.quantile(constriction_ratios, 0.1),
            "log10_cr_median": np.median(constriction_ratios),
            "log10_cr_90": np.quantile(constriction_ratios, 0.9),
        })
    return pd.DataFrame(results)

df_id = df.loc[df["OOD"] == 0]
df_ood = df.loc[df["OOD"] == 1]
bound_table = get_eps_table_from_epsilon(T_list, log10_eps_list, df_ood, upper_quantile)
save_path = os.path.abspath(f"k_given_eps_table_{DOMAIN}{EXPERIMENT_NAME}_DC_quantile_{upper_quantile:.2f}.csv")
bound_table.to_csv(save_path, index=False, sep="\t", encoding="utf-8")
print(f"Saved k_given_eps_table to {save_path}")

# print max ll2_generator and ll2_model for df_id and df_ood
print(f"Max ll2_generator for ID:  {df_id['ll2_generator'].max()}")
print(f"Max ll2_model     for ID:  {df_id['ll2_model'].max()}")
print(f"Max ll2_generator for OOD: {df_ood['ll2_generator'].max()}")
print(f"Max ll2_model     for OOD: {df_ood['ll2_model'].max()}")

# now print the normalized versions:
print(f"Max ll2_generator_norm for ID:  {df_id['ll2_generator_norm'].max()}")
print(f"Max ll2_model_norm     for ID:  {df_id['ll2_model_norm'].max()}")
print(f"Max ll2_generator_norm for OOD: {df_ood['ll2_generator_norm'].max()}")
print(f"Max ll2_model_norm     for OOD: {df_ood['ll2_model_norm'].max()}")

bound_table


# Histogram Log Ratios

In [ ]:

plt.clf()
sns.set_theme(style="whitegrid")
plt.rcParams.update(styles.third)

hue_order = df["Dataset"].unique()
# hue_order = reversed(hue_order)
palette = sns.palettes.color_palette("colorblind", 2)
palette = reversed(palette)
g = sns.histplot(data=df.sort_values("Dataset"), x="log2_ratio_norm", hue="Dataset", stat="density", common_norm=False, bins=50, palette=palette, hue_order=hue_order)
# turn off y axis label and ticks
g.set(ylabel=None)
g.set(yticklabels=[])
# x axis label "Log Likelihood Ratio"
g.set(xlabel="Normalized Log Likelihood Ratio")
# overwrite legend title to empty string
g.get_legend().set_title("")

save_path = os.path.abspath(f"likelihood_ratio_hist_{DOMAIN}{EXPERIMENT_NAME}.pdf")
plt.savefig(save_path, bbox_inches="tight")
print(f"Saved figure to {save_path}")


# Constriction vs OOD Detection

In [ ]:
# get table of detection and certiication metrics

# set the range of k values
k_min = df['log2_ratio_norm'].min() #  -250
k_max = 10 # df['log2_ratio'].max() # 1600

k_list  = np.linspace(k_min, k_max, 100)

metrics = []

# compute the metrics for each k
for k in k_list:
    # get FPR, TPR, Youden's J for threshold k on the log2 ratio
    preds = df['log2_ratio_norm'] > k
    tn, fp, fn, tp = confusion_matrix(df['OOD'], preds).ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0

    #### Calculate the log constriction ratio
    # Apply the get_log_upper_bound_m function with current k
    df["log2_upper_bound_m"] = df.apply(lambda row: get_log_upper_bound_m(row["ll2_generator"], row["n_token_response"], 1, k), axis=1)

    df["log10_upper_bound_m"] = df["log2_upper_bound_m"] / np.log2(10)
    df["log10_upper_bound_f"] = df["ll2_model"] / np.log2(10)

    # Calculate the log10 constriction ratio
    df["log10_constriction_ratio"] = df["log10_upper_bound_f"] - df["log10_upper_bound_m"]

    df_ood = df.loc[df["OOD"] == 1]
    df_id = df.loc[df["OOD"] == 0]

    constriction = df_ood["log10_constriction_ratio"].quantile(0.5)


    metrics.append({
        "k": k,
        "FRR": fpr,
        "TRR": tpr,
        "J": tpr - fpr,
        "log10_constriction_ratio": constriction
    })
metrics = pd.DataFrame(metrics)
# print row of metrics with highest J
print(metrics.loc[metrics["J"].idxmax()])

metrics

In [ ]:
# plot x = Median Log10 Constriction Ratio, y = Youden's J, TPR and FPR
plt.clf()
sns.set_theme(style="whitegrid")
plt.rcParams.update(styles.third)
x_axis = "log10_constriction_ratio"
g = sns.lineplot(data=metrics, x=x_axis, y="J", label="Youden's J")
g = sns.lineplot(data=metrics, x=x_axis, y="TRR", label="TRR")
g = sns.lineplot(data=metrics, x=x_axis, y="FRR", label="FRR")
g.set(xlabel=r"$\text{log}_{10}$ Constriction Ratio (Median)")
g.set(ylabel="")

# Set y-ticks at 0 and 1
g.set_yticks([0, 0.5, 1])

# Move the legend box to the bottom left
g.legend(loc='lower left')

# gray dotted line at x=0
plt.axvline(x=0, color="gray", linestyle="--")
save_path = os.path.abspath(f"constriction_ratio_metrics_{DOMAIN}{EXPERIMENT_NAME}.pdf")
plt.savefig(save_path, bbox_inches="tight")
print(f"Saved figure to {save_path}")


# FRR vs Epsilon-Certificate

In [ ]:
e_log_min = -20 # -350
e_log_max = 0
T = 1

e_log_list = np.linspace(e_log_min, e_log_max, 100)

metrics = []

df_ood = df.loc[df["OOD"] == 1]
g_ood_max = df_ood["ll2_generator"].max()
f_ood_max = df_ood["ll2_model"].max()
f_ood_max_log10 = f_ood_max / np.log2(10)

for e_log in e_log_list:
    k_solution = get_k_given_epsilon(e_log, T, df_ood, -10, 100)
    k = k_solution[1]
    # get FPR and TPR for threshold k on the log2 ratio
    preds = df['log2_ratio_norm'] > k
    tn, fp, fn, tp = confusion_matrix(df['OOD'], preds).ravel()

    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0

    metrics.append({
        "e_log": e_log,
        "FRR": fpr,
        "TRR": tpr,
        "FAR": fnr,
        "J": tpr - fpr
    })

metrics = pd.DataFrame(metrics)

metrics_long = pd.melt(metrics, id_vars=["e_log"], value_vars=["FRR", "TRR", "FAR", "J"], var_name="Metric", value_name="Value")


In [ ]:
sns.set_theme(style="whitegrid")
plt.rcParams.update(styles.third)

x_axis = "e_log"
g = sns.lineplot(data=metrics, x=x_axis, y="FRR", label="FRR")

g.set(ylabel="")
g.legend(loc='lower left')
g.set(xlabel=r"$\text{Log}_{10}$ $\epsilon$-DC")

plt.axvline(x=f_ood_max_log10, color="gray", linestyle="--")
plt.text(f_ood_max_log10-3, 0.6, r"$\max_{\mathcal{D}_{\mathbb{F}}} L(\mathbf{y}|\mathbf{x})$", rotation=90, verticalalignment="center")

save_path = os.path.abspath(f"epsilon_dc_metrics_{DOMAIN}{EXPERIMENT_NAME}.pdf")
plt.savefig(save_path, bbox_inches="tight")
print(f"Saved figure to {save_path}")


# Constriction Ratios

In [ ]:
def print_constriction_table(df, only_ood: bool = True):
    df = df.copy()
    target_distributions = "F(y|x)||G(y)"
    df = df.loc[df["distributions"] == target_distributions]
    df_list = []
    for frr in [0.0, 0.01, 0.05, 0.1, 0.2, 0.25, 0.5]:
        k = get_k_given_frr(df, frr)
        df["log2_upper_bound_m"] = df.apply(lambda x: get_log_upper_bound_m(x["ll2_generator"], x["n_token_response"], 1, k), axis=1)
        df["log10_upper_bound_m"] = df["log2_upper_bound_m"] / np.log2(10)
        df["log10_upper_bound_f"] = df["ll2_model"] / np.log2(10)
        df["log10_constriction_ratio"] = df["log10_upper_bound_f"] - df["log10_upper_bound_m"]
        df_ood = df.loc[df["OOD"] == 1]

        if only_ood:
            median = df_ood.groupby("distributions")["log10_constriction_ratio"].median()
            percentile_10 = df_ood.groupby("distributions")["log10_constriction_ratio"].quantile(0.1)
            percentile_90 = df_ood.groupby("distributions")["log10_constriction_ratio"].quantile(0.9)
        else:
            median = df.groupby("distributions")["log10_constriction_ratio"].median()
            percentile_10 = df.groupby("distributions")["log10_constriction_ratio"].quantile(0.1)
            percentile_90 = df.groupby("distributions")["log10_constriction_ratio"].quantile(0.9)

        # merge to table
        df_upper_bound = pd.concat([percentile_10, median, percentile_90], axis=1)
        df_upper_bound.columns = ["10%", "median", "90%"]

        df_upper_bound["combined"] = df_upper_bound.apply(lambda x: f"{x['10%']:.0f} / {x['median']:.0f} / {x['90%']:.0f}", axis=1)
        df_upper_bound["FRR"] = frr
        df_upper_bound["k"] = k
        df_list.append(df_upper_bound)

    df = pd.concat(df_list)
    save_path = os.path.abspath(f"constriction_table_{DOMAIN}{EXPERIMENT_NAME}.csv")
    df[['FRR', 'k', 'combined']].to_csv(save_path)
    print(f"Saved constriction table to {save_path}")

    print(df)

print_constriction_table(df, True)


In [ ]:
def get_constriction_ratios(df, frr: float, only_ood: bool = True):
    df = df.copy()
    target_distributions = "F(y|x)||G(y)"
    df = df.loc[df["distributions"] == target_distributions]
    k = get_k_given_frr(df, frr)
    df['k'] = k
    df['FRR'] = frr
    df["log2_upper_bound_m"] = df.apply(lambda x: get_log_upper_bound_m(x["ll2_generator"], x["n_token_response"], 1, k), axis=1)
    df["log10_upper_bound_m"] = df["log2_upper_bound_m"] / np.log2(10)
    df["log10_upper_bound_f"] = df["ll2_model"] / np.log2(10)
    df["log10_constriction_ratio"] = df["log10_upper_bound_f"] - df["log10_upper_bound_m"]

    if only_ood:
        return df.loc[df["OOD"] == 1]
    return df

cr_figure_frr = 0.1
df_constriction_ratios = get_constriction_ratios(df, cr_figure_frr, True)

quantiles = [0, 0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99, 1.0]
for q in quantiles:
    print(f"Quantile {q:.2f}: {df_constriction_ratios['log10_constriction_ratio'].quantile(q):.2f}")


In [ ]:
# Set parameters for the plot
# frr_list = [0.1] # Main paper
frr_list = [0, 0.01, 0.05, 0.1, 0.25, 0.50] # Appendix
show_x_axis_label = False # x axis label for last figure is always shown
print_frr = True

# get constriction ratios for each FRR and plot histograms
for cr_figure_frr in frr_list:
    df_constriction_ratios = get_constriction_ratios(df, cr_figure_frr, True)
    print(f'k={df_constriction_ratios["k"].unique().item()}')

    plt.clf()
    sns.set_theme(style="whitegrid")
    plt.rcParams.update(styles.third)

    g = sns.histplot(data=df_constriction_ratios, x="log10_constriction_ratio", hue="Dataset", stat="density", common_norm=False, bins=40, palette="colorblind", alpha=1.0)
    # remove y axis label and ticks
    g.set(ylabel=None)
    g.set(yticklabels=[])

    if not show_x_axis_label and cr_figure_frr != frr_list[-1]:
        g.set(xlabel=None)
    else:
        g.set(xlabel=r"$\text{log}_{10}CR_{k}$ (Log Constriction Ratio)")

    if print_frr:
        # print frr in top right corner
        g.text(0.95, 0.95, f"FRR = {cr_figure_frr:.2f}", horizontalalignment='right', verticalalignment='top', transform=g.transAxes)

    # remove legend
    g.get_legend().remove()

    # x axis limit to 0, max
    min_value = -20
    max_value = 105
    g.set(xlim=(min_value, max_value))
    fraction_below_min = (df_constriction_ratios["log10_constriction_ratio"] < min_value).mean()
    fraction_above_max = (df_constriction_ratios["log10_constriction_ratio"] > max_value).mean()
    if max_value is not None and fraction_above_max > 0.01:
            rich.print(f"[bold yellow]Warning:[/bold yellow] 1% of Log10 constriction ratios exceeds {max_value}.")
    if min_value is not None and fraction_below_min > 0.01:
        rich.print(f"[bold yellow]Warning:[/bold yellow] 1% of Log10 constriction ratios is below {min_value}.")


    # vertical bar at x=0
    plt.axvline(x=0, color="gray", linestyle="--")

    save_path = os.path.abspath(f"cr_histograms/cr_hist_frr_{cr_figure_frr:.2f}_{DOMAIN}{EXPERIMENT_NAME}.pdf")
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path, bbox_inches="tight")
    print(f"Saved figure to {save_path}")



# Atomic Certificates

In [ ]:
def get_atomic_certificate_values(df, T, FRR):
    df = df.copy()
    results = []
    k = get_k_given_frr(df, FRR)
    print(f"k={k}")

    df["log2_upper_bound_m"] = df.apply(lambda x: get_log_upper_bound_m(x["ll2_generator"], x["n_token_response"], 1, k), axis=1)
    df["log10_upper_bound_m"] = df["log2_upper_bound_m"] / np.log2(10)
    df["log10_upper_bound_f"] = df["ll2_model"] / np.log2(10)
    df["upper_bound_m"] = 2 ** df["log2_upper_bound_m"].astype(np.float128)
    # set values above 10000 to nan. and values below 1e-300 to nan
    df["upper_bound_m"] = df["upper_bound_m"].apply(lambda x: x if (x > 1e-300 and x < 10000) else 2e-300)
    print(df["upper_bound_m"].min())
    print(df["upper_bound_m"].max())
    results = df[["log10_upper_bound_m", "upper_bound_m", "log10_upper_bound_f", "Dataset", "n_token_response"]]
    return results

ac_ecdf_figure_frr = 0.1

df_ac_values = get_atomic_certificate_values(df, 1, ac_ecdf_figure_frr)


In [ ]:
print_legend = True
y_axis_label_and_ticks = False

plt.clf()
sns.set_theme(style="whitegrid")
plt.rcParams.update(styles.third)

hue_order = ['Target Domain $\\mathcal{D}_{T}$', 'Other $\\mathcal{D}_{F}$']

palette = sns.color_palette()[:2][::-1]

g = sns.ecdfplot(data=df_ac_values, x="log10_upper_bound_m", hue="Dataset", hue_order=hue_order, palette=palette)

# set x limits
g.set(xlim=(None, 10))
g.set(xlabel=r"$\text{log}_{10}$ $\epsilon_{\mathbf{y}}$-AC")
g.set_title("Medical QA", fontsize=8, pad=2)

g.set(ylabel=None)
if not y_axis_label_and_ticks:
    g.set(yticklabels=[])

if print_legend:
    legend = g.get_legend()

    # Move the legend to the exact corner
    legend.set_bbox_to_anchor((-0.04, 1.06))  # Top-left corner of the axis
    legend.set_loc("upper left")  # Align to the upper left corner
    # Remove legend title
    legend.set_title("")
    # Customize legend line width and reduce padding
    for line in legend.get_lines():
        line.set_linewidth(2)  # Adjust line width

    # Adjust legend spacing
    legend.set_frame_on(False)  # Remove the legend frame for a cleaner look
else:
    g.get_legend().remove()

save_path = os.path.abspath(f"epsilon_ac_ecdf_frr_{ac_ecdf_figure_frr:.2f}_{DOMAIN}{EXPERIMENT_NAME}.pdf")
plt.savefig(save_path, bbox_inches="tight")
print(f"Saved figure to {save_path}")
